```
PS C:\Users\Erik\Desktop\GitHub\Atmel-SAML11\Python\pydgilib\pydgilib> ..\..\..\..\..\..\Miniconda3\envs\SAML11-32\python.exe

from dgilib import DGILib
with DGILib(verbose=3) as dgilib:
    dgilib.get_build_number()

```

- Goto folder with setup.py
- run pip install -e .
- in python:

```
from pydgilib import DGILib
with DGILib(verbose=3) as dgilib:
    dgilib.get_build_number()
```

In [1]:
from pydgilib import *

In [2]:
from time import sleep

In [3]:
from pydgilib.dgilib_exceptions import *


class PowerStatusError(Error):
    """Exception raised when checking `auxiliary_power_get_status()`"""

    pass


class PowerReadError(Error):
    """Exception raised when reading power buffer"""

    pass

In [4]:
"""This module provides Python bindings for DGILibExtra GPIO interface."""

__author__ = "Erik Wouters <ehwo(at)kth.se>"
__credits__ = "Atmel Corporation. / Rev.: Atmel-42771A-DGILib_User Guide-09/2016"
__license__ = "MIT"
__version__ = "0.1"
__revision__ = " $Id: dgilib_interface_gpio.py 1586 2019-02-13 15:56:25Z EWouters $ "
__docformat__ = "reStructuredText"


# TODO: make these functions faster
def int2bool(i):
    """Convert int to list of bool
    """
    return [bit is "1" for bit in f"{i:04b}"]


def bool2int(b):
    """Convert list of bool to int
    """
    return int("".join("1" if d else "0" for d in b), 2)


class DGILibInterfaceGPIO(object):
    """Python bindings for DGILib GPIO interface.

    The GPIO interface consists of four lines available, which can be individually set to input or output
    through the configuration interface. This interface can only be used in Timestamp mode. Input lines are
    monitored and will trigger an entry to be added to the timestamp buffer on each change. Output lines can
    be controlled through the send data command.
    
    ## Parsing
    Each received data byte corresponds to an input pattern on the GPIO pins. If a bit is 1 it means that the
    corresponding GPIO pin is high, a 0 means a low level.
    
    ## Configuration
    The GPIO configuration controls the direction of the pins.
    Field ID Description
    Input pins 0 Setting a bit to 1 means the pin is monitored.
    Output pins 1 Setting a bit to 1 means the pin is set to output and can be controlled by the send
    command.
    """

    def __init__(self, *args, **kwargs):
        """
        :Example:

        >>> with DGILibExtra(read_mode=[True] * 4) as dgilib:
        ...     dgilib.get_major_version()
        5
        """

        # Argument parsing
        self.read_mode = kwargs.get("read_mode", [False] * 4)
        self.write_mode = kwargs.get("write_mode", [False] * 4)
        if self.verbose:
            print("read_mode: ", self.read_mode)
            print("write_mode: ", self.write_mode)

    def __enter__(self):
        """
        """
        
        self.gpio_set_config(read_mode=self.read_mode, write_mode=self.write_mode)

        return self

    def __exit__(self, exc_type, exc_value, traceback):
        """
        """
        
        self.gpio_set_config()  # Disables interface.

    def gpio_get_config(self):
        """Get the pin-mode for the GPIO pins
        
        The GPIO configuration controls the direction of the pins.
        
        Input pins:  Setting a bit to 1 means the pin is monitored.
        Output pins: Setting a bit to 1 means the pin is set to output and can be controlled by the send
        command.
        
        :return: Tuple of:
            - List of read modes, Setting a pin to True means the pin is monitored.
            - List of write modes, Setting a pin to True means the pin is set to output and can be controlled by the send command.
        :rtype: (list(bool), list(bool))
        """

        # Get the configuration
        config_id, config_value = self.interface_get_configuration(INTERFACE_GPIO)

        # Convert int to lists of bool
        read_mode = int2bool(config_value[0])
        write_mode = int2bool(config_value[1])

        return read_mode, write_mode

#     def gpio_set_config(self, read_mode=[False] * 4, write_mode=[False] * 4):
        
#         # Update internal values
#         self.read_mode = read_mode
#         self.write_mode = write_mode
        
    def gpio_set_config(self, **kwargs):
        """Set the pin-mode for the GPIO pins
        
        The GPIO configuration controls the direction of the pins, and enables the interface if needed.
        
        Input pins:  Setting a bit to 1 means the pin is monitored.
        Output pins: Setting a bit to 1 means the pin is set to output and can be controlled by the send
        command.
        
        If any of the pins are set to read mode or write mode the GPIO interface will be enabled. If none of the pins are set to read mode
        or write mode the GPIO interface will be disabled.
        
        :param read_mode: List of modes, Setting a pin to True means the pin is monitored.
        :type read_mode: list(bool)
        :param write_mode: List of modes, Setting a pin to True means the pin is set to output and can be controlled by the send command.
        :type write_mode: list(bool)
        """
        
        # Argument parsing
        self.read_mode = kwargs.get("read_mode", self.read_mode)
        self.write_mode = kwargs.get("write_mode", self.write_mode)

        # Convert lists of bool to int
        read_mode = bool2int(self.read_mode)
        write_mode = bool2int(self.write_mode)

        # Set the configuration
        if "read_mode" in kwargs:
            self.interface_set_configuration(INTERFACE_GPIO, [0], [read_mode])
        if "write_mode" in kwargs:
            self.interface_set_configuration(INTERFACE_GPIO, [1], [write_mode])

        # Enable the interface if any of the pins are set to read mode or write mode
        if read_mode + write_mode:
            if not INTERFACE_GPIO in self.enabled_interfaces:
                self.interface_enable(INTERFACE_GPIO)
                self.enabled_interfaces.append(INTERFACE_GPIO)
            if self.timer_factor is None:
                self.timer_factor = self.get_time_factor()
        elif INTERFACE_GPIO in self.enabled_interfaces:  # Disable the interface if none of the pins are set to read mode or write mode
            self.interface_disable(INTERFACE_GPIO)
            self.enabled_interfaces.remove(INTERFACE_GPIO)

    def gpio_read(self):
        """Get the state of the GPIO pins.
        
        Clears the buffer and returns the values.
        
        :return: Tuple of list of list of pin states (bool) and list of timestamps in seconds
        :rtype: (list(list(bool)), list(float)
        """

        # Read the data from the buffer
        pin_values, ticks = self.interface_read_data(INTERFACE_GPIO)

        pin_values = [int2bool(pin_value) for pin_value in pin_values]
        timestamps = [tick * self.timer_factor for tick in ticks]

        return pin_values, timestamps

    def gpio_write(self, pin_values):
        """Set the state of the GPIO pins
        
        Make sure to set the pin to write mode first. Possibly also needs to be configured properly on the board
        
        A maximum of 255 elements can be written each time. An error return code will be given if data hasn’t been written yet.
        
        :param pin_values: List of pin values. Has to include all four pins ? TODO: TEST
        :type pin_values: list(bool)
        """

        # Convert list of bool to int
        pin_values = bool2int(pin_values)

        self.interface_write_data(INTERFACE_GPIO, [pin_values])

In [5]:
"""This module provides Python bindings for DGILibExtra Logger."""

__author__ = "Erik Wouters <ehwo(at)kth.se>"
__credits__ = "Atmel Corporation. / Rev.: Atmel-42771A-DGILib_User Guide-09/2016"
__license__ = "MIT"
__version__ = "0.1"
__revision__ = " $Id: dgilib_logger.py 1586 2019-02-13 15:56:25Z EWouters $ "
__docformat__ = "reStructuredText"

import csv

class DGILibLogger(object):
    """Python bindings for DGILib Logger.
    """

    def __init__(self, *args, **kwargs):
        """
        """
        
        # Argument parsing
#         interfaces
#             - GPIO mode
#             - Power mode
        
#         file_name
#         plot
        
#         update_callback
        
        self.interfaces   = kwargs.get("interfaces",[])  # List of interfaces (ints)
#         self.interface_configs = {}
#         if INTERFACE_GPIO in self.interfaces:
#             self.interface_configs[INTERFACE_GPIO]   = kwargs.get("interface_configs",[])  # List of interfaces (ints)
        self.file_name    = kwargs.get("file_name",None)  # File name (passed to open), if empty no logging will be done.
        self.data_in_obj  = kwargs.get("data_in_obj",False)  # if this is true the data will be available through self.timestamp[INTERFACE] and self.data[INTERFACE]
        

#         output = csv.writer(open('export_log.csv', 'w'))

#         for foo in bar:
#            # do work
#            output.writerow([status, view, filename, out, err, current_time])
        
        
#         self.options = {
#             'option1' : 'default_value1',
#             'option2' : 'default_value2',
#             'option3' : 'default_value3', }

#         self.options.update(kwargs)
#         print self.options
        
    def __enter__(self):
        """
        """
        
        if self.file_name is not None:
            # TODO per interface
            self.writer = "csv.writer"(open(file_name, 'w'))
        
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        """
        """
        
            

    def start(self):
        self.start_polling()
        self.auxiliary_power_start()
        
    def stop(self):
        dgilib.stop_polling()
        dgilib.auxiliary_power_stop()

    def logger(self, duration=10):
        """logger
        """
        
        
#         self.writer[INTERFACE_GPIO].writerow(['status', 'view', 'filename', 'stdout', 'stderr', 'time'])

        self.start()
        sleep(duration)
        power_data = self.power_read()
        gpio_data = self.gpio_read()

        self.stop()

        return power_data[0], gpio_data

In [6]:
"""This module provides Python bindings for DGILibExtra Power interface."""

__author__ = "Erik Wouters <ehwo(at)kth.se>"
__credits__ = "Atmel Corporation. / Rev.: Atmel-42771A-DGILib_User Guide-09/2016"
__license__ = "MIT"
__version__ = "0.1"
__revision__ = " $Id: dgilib_interface_power.py 1586 2019-02-13 15:56:25Z EWouters $ "
__docformat__ = "reStructuredText"


class DGILibInterfacePower(object):
    """Python bindings for DGILib Power interface.

    """

    def __init__(self, *args, **kwargs):
        """
        :Example:

        >>> pb = [{"channel": 0, "power_type": 0}]
        >>> with DGILibExtra(power_buffers=pb) as dgilib:
        ...     dgilib.get_major_version()
        5
        """

        # Argument parsing
        self.power_buffers = kwargs.get("power_buffers", [])
        if self.verbose:
            print("power_buffers: ", self.power_buffers)

    def __enter__(self):
        """
        """

        # Check if calibration is valid and trigger calibration if it is not
        self.circuit_type = self.auxiliary_power_get_circuit_type()
        if not self.auxiliary_power_calibration_is_valid():
            self.auxiliary_power_trigger_calibration(self.circuit_type)

        # Register buffers inside the library for the buffers specified in self.power_buffers
        for power_buffer in self.power_buffers:
            self.auxiliary_power_register_buffer_pointers(
                channel=power_buffer["channel"], power_type=power_buffer["power_type"]
            )

        return self

    def __exit__(self, exc_type, exc_value, traceback):
        """
        """
        self.power_set_config([])

    def auxiliary_power_calibration(self, circuit_type=XAM):
        """Calibrate the Auxilary Power interface of the device.
        
        :param circuit_type: Type of calibration to trigger (defaults to XAM)
        :type circuit_type: int
        """

        self.auxiliary_power_trigger_calibration(circuit_type)
        while self.auxiliary_power_get_status() == CALIBRATING:
            sleep(0.1)

    def power_get_config(self):
        """Get the power config options.
        
        :return: Power buffers configuration list of dictionaries like [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}]
        :rtype: list(dict())
        """

        # Return the configuration
        return self.power_buffers

    def power_set_config(self, power_buffers):
        """Set the power config options.
        
        Register buffers inside the library for the buffers specified in power_buffers and removes ones that are not present.
        
        :param power_buffers: Power buffers configuration list of dictionaries like [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}]
        :type power_buffers: list(dict())
        """

        # Disable the configurations that are not in the new config and remove them from self.power_buffers
        for power_buffer in self.power_buffers:
            if power_buffer not in power_buffers:
                self.auxiliary_power_unregister_buffer_pointers(
                    channel=power_buffer["channel"],
                    power_type=power_buffer["power_type"],
                )
                self.power_buffers.remove(power_buffer)

        # Enable the configurations that are in the new config and not in self.power_buffers
        for power_buffer in power_buffers:
            if power_buffer not in self.power_buffers:
                self.auxiliary_power_register_buffer_pointers(
                    channel=power_buffer["channel"],
                    power_type=power_buffer["power_type"],
                )
                self.power_buffers.append(power_buffer)

    def power_read_buffer(self, power_buffer, *args, **kwargs):
        """Read power data of the specified buffer.
        
        TODO: Copies parsed power data into the specified buffer. Remember to lock the buffers first. If the count parameter is the same as max_count there is probably more data to be read. Do another read to get the remaining data.
        
        :return: Tuple of list of power samples in Ampere and list of timestamps in seconds
        :rtype: (list(float), list(float))
        """

        # Check if power_buffer is in self.power_buffers
        if power_buffer not in self.power_buffers:
            raise PowerReadError(
                f"Power Buffer {power_buffer} does not exist in self.power_buffers: {self.power_buffers}."
            )

        # Check if auxiliary_power_get_status() is in IDLE = 0x00, RUNNING = 0x01, DONE = 0x02 or OVERFLOWED = 0x11
        # and raise DevicePowerStatusError if it is.
        power_status = self.auxiliary_power_get_status()
        if self.verbose:
            print(f"power_status: {power_status}")
        # if power_status <= DONE or power_status == OVERFLOWED:
        if power_status not in (IDLE, RUNNING, DONE, OVERFLOWED):
            raise DevicePowerStatusError(f"Power Status {power_status}.")
        if power_status == OVERFLOWED:
            print(
                f"BUFFER OVERFLOW, call this function more frequently or increase the buffer size."
            )

        # Create variable to the store data in
        power_samples = []
        timestamps = []

        # TODO: Check implementation in case of buffer overflow.
        #  Should auxiliary_power_lock_data_for_reading() be inside the loop or before?

        # Get the data from the buffer in the library
        while True:
            self.auxiliary_power_lock_data_for_reading()
            data = self.auxiliary_power_copy_data(
                power_buffer["channel"], power_buffer["power_type"], *args, **kwargs
            )
            power_samples.append(data[0])
            timestamps.append(data[1])
            self.auxiliary_power_free_data()  # BUG: This probably clears all channels!
            # Repeat the loop untill there is no buffer overflow (which should always be avoided.)
            if self.auxiliary_power_get_status() != OVERFLOWED:
                break

        if self.verbose >= 4:
            print(power_samples, timestamps)
        return power_samples, timestamps

    def power_read(self, *args, **kwargs):
        """Read power data from all enabled buffers.
        
        The returned list has the same indexes as the list obtained from `power_get_config()`
        
        :return: List of tuples of list of power samples in Ampere and list of timestamps in seconds
        :rtype: list[(list(float), list(float)), ...]
        """

        # Check if auxiliary_power_get_status() is in IDLE = 0x00, RUNNING = 0x01, DONE = 0x02 or OVERFLOWED = 0x11
        # and raise DevicePowerStatusError if it is.
        power_status = self.auxiliary_power_get_status()
        if self.verbose:
            print(f"power_status: {power_status}")
        # if power_status <= DONE or power_status == OVERFLOWED:
        if power_status not in (IDLE, RUNNING, DONE, OVERFLOWED):
            raise DevicePowerStatusError(f"Power Status {power_status}.")
        if power_status == OVERFLOWED:
            raise DevicePowerStatusError(
                f"BUFFER OVERFLOW, call `power_read()` more frequently or increase the buffer size."
            )

        # Create variable to the store data in
        data = []

        # Get the data from the buffer in the library
        self.auxiliary_power_lock_data_for_reading()
        for power_buffer in self.power_buffers:
            data.append(
                self.auxiliary_power_copy_data(
                    power_buffer["channel"], power_buffer["power_type"], *args, **kwargs
                )
            )
        self.auxiliary_power_free_data()

        #         # TODO: Check implementation in case of buffer overflow.
        #         #  Should auxiliary_power_lock_data_for_reading() be inside the loop or before?

        #         # Get the data from the buffer in the library
        #         while True:
        #             self.auxiliary_power_lock_data_for_reading()
        #             for power_buffer in self.power_buffers:
        #                 data = self.auxiliary_power_copy_data(power_buffer["channel"], power_buffer["power_type"], *args, **kwargs)
        #                 power_samples.append(data[0])
        #                 timestamps.append(data[1])
        #             self.auxiliary_power_free_data()
        #             # Repeat the loop untill there is no buffer overflow (which should always be avoided.)
        #             if self.auxiliary_power_get_status() != OVERFLOWED:
        #                 break

        if self.verbose >= 4:
            print(data)
        return data


#     dgilib.auxiliary_power_register_buffer_pointers()
#     dgilib.start_polling()
#     dgilib.auxiliary_power_start()
#     dgilib.auxiliary_power_get_status()
#     sleep(1)
#     dgilib.auxiliary_power_lock_data_for_reading()
#     dat = dgilib.auxiliary_power_copy_data()
#     dgilib.auxiliary_power_free_data()
#     dgilib.auxiliary_power_stop()
#     dgilib.auxiliary_power_get_status()

In [7]:
"""This module provides Python bindings for DGILib Extra."""

__author__ = "Erik Wouters <ehwo(at)kth.se>"
__credits__ = "Atmel Corporation. / Rev.: Atmel-42771A-DGILib_User Guide-09/2016"
__license__ = "MIT"
__version__ = "0.1"
__revision__ = " $Id: dgilib_extra.py 1586 2019-02-13 15:56:25Z EWouters $ "
__docformat__ = "reStructuredText"

# from dgilib_interface_gpio import DGILibInterfaceGPIO
# from dgilib_extra import DGILibLogger


class DGILibExtra(DGILib, DGILibInterfaceGPIO, DGILibInterfacePower, DGILibLogger):
    """Python bindings for DGILib Extra.
    """

    def __init__(self, *args, **kwargs):
        """
        """

        DGILib.__init__(self, *args, **kwargs)
        
        DGILibInterfaceGPIO.__init__(self, *args, **kwargs)
        DGILibInterfacePower.__init__(self, *args, **kwargs)
        DGILibLogger.__init__(self, *args, **kwargs)

        self.available_interfaces = []
        self.enabled_interfaces = []
        self.timer_factor = None

    def __enter__(self):
        """
        """

        DGILib.__enter__(self)

        self.available_interfaces = self.interface_list()
        
        DGILibInterfaceGPIO.__enter__(self)
        DGILibInterfacePower.__enter__(self)

        DGILibLogger.__enter__(self)

        return self

    def __exit__(self, exc_type, exc_value, traceback):

        DGILibLogger.__exit__(self, exc_type, exc_value, traceback)
        # DGILibInterfaceGPIO.__exit__(self, exc_type, exc_value, traceback)
        
        DGILibInterfacePower.__exit__(self, exc_type, exc_value, traceback)

        for interface in self.enabled_interfaces:
            self.interface_disable(interface)

        DGILib.__exit__(self, exc_type, exc_value, traceback)

        if self.verbose:
            print("bye from DGILib Extra")

    def info(self):
        """Get the build information of DGILib.
        
        :param print_info: A flag used to print the build information to the console (default is False)
        :type print_info: bool
        :return:  Version information of DGILib:
            - major_version: the major_version of DGILib
            - minor_version: the minor_version of DGILib
            - build_number: the build number of DGILib. 0 if not supported
            - major_fw: the major firmware version of the DGI device connected
            - minor_fw: the minor firmware version of the DGI device connected
        :rtype: tuple
        """

        major_version = self.get_major_version()
        minor_version = self.get_minor_version()
        build_number = self.get_build_number()
        major_fw, minor_fw = self.get_fw_version()

        return major_version, minor_version, build_number, major_fw, minor_fw

    def device_reset(self, duration=1):
        """Set the device reset line for duration seconds.
        """

        dgilib.target_reset(True)
        sleep(duration)
        dgilib.target_reset(False)

    def get_time_factor(self):
        """Get the factor to multiply timestamps by to get seconds.
        
        :return: timer_factor
        :rtype: double
        """

        _, config_value = self.interface_get_configuration(INTERFACE_TIMESTAMP)
        timer_prescaler = config_value[0]
        timer_frequency = config_value[1]

        if self.verbose:
            print(
                f"timer_factor: {timer_prescaler / timer_frequency}, timer_prescaler: {timer_prescaler}, timer_frequency: {timer_frequency}"
            )

        return timer_prescaler / timer_frequency

In [8]:
with DGILibExtra(read_mode=[True] * 4) as dgilib:
    dgilib.get_major_version()

read_mode:  [True, True, True, True]
write_mode:  [False, False, False, False]
power_buffers:  []
device_count: 1
	0 get_device_serial: b'ATML3138061800001604'
	0 connect
connection_status: 0
	0 auxiliary_power_initialize
	0 interface_list: [0, 32, 34, 48, 64, 65, 80], interfaceCount: 7
	0 interface_set_configuration: 48, config_cnt: 1
	0 interface_set_configuration: 48, config_cnt: 1
	0 interface_enable: 48, timestamp: True
	0 interface_get_configuration: 0, config_cnt: 2
timer_factor: 5.333333333333333e-07, timer_prescaler: 32, timer_frequency: 60000000
	0 auxiliary_power_get_circuit_type: 16
auxiliary_power_calibration_is_valid: 1
major_version: 5
	0 interface_disable: 48
	0 auxiliary_power_uninitialize
bye from Auxiliary
	0 disconnect
bye from Housekeeping
bye from Discovery
bye from DGILib
bye from DGILib Extra


In [9]:
with DGILibExtra() as dgilib:
     dgilib.get_major_version()

read_mode:  [False, False, False, False]
write_mode:  [False, False, False, False]
power_buffers:  []
device_count: 1
	0 get_device_serial: b'ATML3138061800001604'
	0 connect
connection_status: 0
	0 auxiliary_power_initialize
	0 interface_list: [0, 32, 34, 48, 64, 65, 80], interfaceCount: 7
	0 interface_set_configuration: 48, config_cnt: 1
	0 interface_set_configuration: 48, config_cnt: 1
	0 auxiliary_power_get_circuit_type: 16
auxiliary_power_calibration_is_valid: 1
major_version: 5
	0 auxiliary_power_uninitialize
bye from Auxiliary
	0 disconnect
bye from Housekeeping
bye from Discovery
bye from DGILib
bye from DGILib Extra


In [10]:
pb = [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}]
power_data = []
with DGILibExtra(verbose=0, power_buffers=pb) as dgilib:
    dgilib.power_get_config()
    dgilib.start_polling()
    dgilib.auxiliary_power_start()
    dgilib.auxiliary_power_get_status()
    #     for i in range(10):
    #         sleep(1)
    #         power_data += dgilib.power_read()  # This is definitely not the way to combine these lists. Should probably define a class and append method.
    #         print(len(data[0][0]))
    sleep(1)
    power_data = dgilib.power_read()
    dgilib.auxiliary_power_stop()
    dgilib.auxiliary_power_get_status()

    dgilib.stop_polling()

In [11]:
power_data[0][0][:10], power_data[0][1][:10]

([0.004130123183131218,
  0.0034313094802200794,
  0.003787968773394823,
  0.0038863576482981443,
  0.003787968773394823,
  0.0038863576482981443,
  0.003787968773394823,
  0.0038863576482981443,
  0.0038002673536539078,
  0.0038740590680390596],
 [0.006832130133333334,
  0.006898133333333334,
  0.006964136533333334,
  0.007030139733333334,
  0.007096142933333334,
  0.007162146133333334,
  0.007228149333333334,
  0.007294152533333334,
  0.0073601557333333335,
  0.007426158933333334])

In [12]:
pb = [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}]
power_data = []
gpio_data = []
with DGILibExtra(verbose=0, power_buffers=pb) as dgilib:
    dgilib.gpio_set_config(read_mode=[True, True, True, True], write_mode=[False, False, False, False])
    dgilib.start_polling()
    dgilib.auxiliary_power_start()
    dgilib.auxiliary_power_get_status()
    sleep(1)
    power_data = dgilib.power_read()
    gpio_data = dgilib.gpio_read()
    dgilib.auxiliary_power_stop()
    dgilib.auxiliary_power_get_status()

    dgilib.stop_polling()

In [13]:
power_data[0][0][:10], power_data[0][1][:10], gpio_data[0][:10], gpio_data[1][:10]

([0.003763001412153244,
  0.003382114926353097,
  0.0038740590680390596,
  0.0037756701931357384,
  0.0038863576482981443,
  0.0037756701931357384,
  0.003898656228557229,
  0.0037756701931357384,
  0.0038863576482981443,
  0.0038002673536539078],
 [0.008059863466666676,
  0.008125866666666676,
  0.008191869866666675,
  0.008257873066666676,
  0.008323876266666676,
  0.008389879466666675,
  0.008455882666666676,
  0.008521885866666676,
  0.008587889066666675,
  0.008653892266666676],
 [[True, True, False, False],
  [True, True, False, True],
  [True, True, True, True],
  [True, True, True, False],
  [True, True, False, False],
  [True, True, False, True],
  [True, True, True, True],
  [True, True, True, False],
  [True, True, False, False],
  [True, True, False, True]],
 [0.001981333333333333,
  0.0599936,
  0.07014133333333333,
  0.19115946666666667,
  0.2013072,
  0.32232320000000003,
  0.33247093333333333,
  0.453488,
  0.4636352,
  0.5846490666666667])

In [14]:
with DGILibExtra(verbose=2) as dgilib:
    print(dgilib.available_interfaces)

read_mode:  [False, False, False, False]
write_mode:  [False, False, False, False]
power_buffers:  []
device_count: 1
	0 get_device_serial: b'ATML3138061800001604'
	0 connect
connection_status: 0
	0 auxiliary_power_initialize
	0 interface_list: [0, 32, 34, 48, 64, 65, 80], interfaceCount: 7
	0 interface_set_configuration: 48, config_cnt: 1
		config_id: 0,	value: 0
	0 interface_set_configuration: 48, config_cnt: 1
		config_id: 1,	value: 0
	0 auxiliary_power_get_circuit_type: 16
auxiliary_power_calibration_is_valid: 1
[0, 32, 34, 48, 64, 65, 80]
	0 auxiliary_power_uninitialize
bye from Auxiliary
	0 disconnect
bye from Housekeeping
bye from Discovery
bye from DGILib
bye from DGILib Extra


In [15]:
pin_values = []
timestamps = []
gpio_data = [pin_values, timestamps]

write_values = [True, False, False, False]

with DGILibExtra(verbose=2) as dgilib:
    config = dgilib.gpio_get_config()
    print(config)
#     dgilib.gpio_set_config(read_mode=[True, True, False, False], write_mode=[True, False, True, False])
    dgilib.gpio_set_config(read_mode=[False, False, False, False], write_mode=[True, True, True, True])
    config = dgilib.gpio_get_config()
    print(config)
    dgilib.start_polling()
    sleep(1)
    for i in range(10):
        print(write_values[i%4:]+write_values[:i%4])
        dgilib.gpio_write(write_values)
        pin_values, timestamps = dgilib.gpio_read()
        gpio_data[0] += pin_values
        gpio_data[1] += timestamps
        sleep(1)
    dgilib.stop_polling()
    pin_values, timestamps = dgilib.gpio_read()
    gpio_data[0] += pin_values
    gpio_data[1] += timestamps
print(gpio_data)
print(len(gpio_data[0]), len(gpio_data[1]))

read_mode:  [False, False, False, False]
write_mode:  [False, False, False, False]
power_buffers:  []
device_count: 1
	0 get_device_serial: b'ATML3138061800001604'
	0 connect
connection_status: 0
	0 auxiliary_power_initialize
	0 interface_list: [0, 32, 34, 48, 64, 65, 80], interfaceCount: 7
	0 interface_set_configuration: 48, config_cnt: 1
		config_id: 0,	value: 0
	0 interface_set_configuration: 48, config_cnt: 1
		config_id: 1,	value: 0
	0 auxiliary_power_get_circuit_type: 16
auxiliary_power_calibration_is_valid: 1
	0 interface_get_configuration: 48, config_cnt: 2
		config_id: 0,	value: 0
		config_id: 1,	value: 0
([False, False, False, False], [False, False, False, False])
	0 interface_set_configuration: 48, config_cnt: 1
		config_id: 0,	value: 0
	0 interface_set_configuration: 48, config_cnt: 1
		config_id: 1,	value: 15
	0 interface_enable: 48, timestamp: True
	0 interface_get_configuration: 0, config_cnt: 2
		config_id: 0,	value: 32
		config_id: 1,	value: 60000000
timer_factor: 5.33

In [16]:
pin_values = []
timestamps = []
gpio_data = [pin_values, timestamps]
with DGILibExtra(verbose=2) as dgilib:
    config = dgilib.gpio_get_config()
    print(config)
#     dgilib.gpio_set_config(read_mode=[True, True, False, False], write_mode=[True, False, True, False])
    dgilib.gpio_set_config(read_mode=[True, True, True, True], write_mode=[False, False, False, False])
    config = dgilib.gpio_get_config()
    print(config)
    dgilib.start_polling()
    sleep(1)
    for i in range(10):
        pin_values, timestamps = dgilib.gpio_read()
        gpio_data[0] += pin_values
        gpio_data[1] += timestamps
        sleep(1)
    dgilib.stop_polling()
    pin_values, timestamps = dgilib.gpio_read()
    gpio_data[0] += pin_values
    gpio_data[1] += timestamps
print(gpio_data)
print(len(gpio_data[0]), len(gpio_data[1]))

read_mode:  [False, False, False, False]
write_mode:  [False, False, False, False]
power_buffers:  []
device_count: 1
	0 get_device_serial: b'ATML3138061800001604'
	0 connect
connection_status: 0
	0 auxiliary_power_initialize
	0 interface_list: [0, 32, 34, 48, 64, 65, 80], interfaceCount: 7
	0 interface_set_configuration: 48, config_cnt: 1
		config_id: 0,	value: 0
	0 interface_set_configuration: 48, config_cnt: 1
		config_id: 1,	value: 0
	0 auxiliary_power_get_circuit_type: 16
auxiliary_power_calibration_is_valid: 1
	0 interface_get_configuration: 48, config_cnt: 2
		config_id: 0,	value: 0
		config_id: 1,	value: 0
([False, False, False, False], [False, False, False, False])
	0 interface_set_configuration: 48, config_cnt: 1
		config_id: 0,	value: 15
	0 interface_set_configuration: 48, config_cnt: 1
		config_id: 1,	value: 0
	0 interface_enable: 48, timestamp: True
	0 interface_get_configuration: 0, config_cnt: 2
		config_id: 0,	value: 32
		config_id: 1,	value: 60000000
timer_factor: 5.33

In [17]:
with DGILibExtra(verbose=1) as dgilib:
    dgilib.get_time_factor()
    dgilib.auxiliary_power_calibration()

read_mode:  [False, False, False, False]
write_mode:  [False, False, False, False]
power_buffers:  []
device_count: 1
	0 get_device_serial: b'ATML3138061800001604'
	0 connect
connection_status: 0
	0 auxiliary_power_initialize
	0 interface_list: [0, 32, 34, 48, 64, 65, 80], interfaceCount: 7
	0 interface_set_configuration: 48, config_cnt: 1
	0 interface_set_configuration: 48, config_cnt: 1
	0 auxiliary_power_get_circuit_type: 16
auxiliary_power_calibration_is_valid: 1
	0 interface_get_configuration: 0, config_cnt: 2
timer_factor: 5.333333333333333e-07, timer_prescaler: 32, timer_frequency: 60000000
	0 auxiliary_power_trigger_calibration
power_status: 3
power_status: 3
power_status: 2
	0 auxiliary_power_uninitialize
bye from Auxiliary
	0 disconnect
bye from Housekeeping
bye from Discovery
bye from DGILib
bye from DGILib Extra


In [18]:
# with DGILibExtra(verbose=1) as dgilib:
#     dgilib.GPIO(read_mode=[True, True, False, False])
#     dgilib.Power(channels=[True, True])
#     dgilib.start(file/plot/object)
#     dgilib.stop()

In [19]:
data = []

config_dict = {
    "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
    "read_mode": [True, True, True, True],
    "write_mode": [False, False, False, False],
    "verbose": 0,
}

with DGILibExtra(**config_dict) as dgilib:
    data = dgilib.logger(1)

In [20]:
%matplotlib ipympl

In [21]:
import matplotlib.pyplot as plt
# import numpy as np

In [22]:
plt.plot(data[0][1], data[0][0])
plt.show()

C:\Users\Erik\Miniconda3\envs\SAML11-32\lib\site-packages\matplotlib\__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [23]:
plt.figure()
for pin in range(4):
    plt.plot(data[1][1], [d[pin] for d in data[1][0]])
plt.show()

C:\Users\Erik\Miniconda3\envs\SAML11-32\lib\site-packages\matplotlib\__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [24]:
plt.figure()
plt.plot(data[0][1], data[0][0])
max_data = max(data[0][0])
for pin in range(4):
    plt.plot(data[1][1], [d[pin]*max_data for d in data[1][0]])
plt.show()

C:\Users\Erik\Miniconda3\envs\SAML11-32\lib\site-packages\matplotlib\__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [25]:
data[0][1][:10], data[0][0][:10], data[1][1][:10], data[1][0][:10]

([0.015229465066666673,
  0.015295466666666674,
  0.015361468266666675,
  0.015427469866666673,
  0.015493471466666674,
  0.015559473066666675,
  0.015625474666666674,
  0.015691476266666674,
  0.015757477866666675,
  0.015823479466666675],
 [0.004221904091536999,
  0.003382114926353097,
  0.0038125659339129925,
  0.0038002673536539078,
  0.0038494619075208902,
  0.003787968773394823,
  0.0038494619075208902,
  0.003787968773394823,
  0.0038371633272618055,
  0.003787968773394823],
 [0.005010666666666667,
  0.05832586666666667,
  0.06847626666666667,
  0.18950453333333334,
  0.19965386666666668,
  0.32067840000000003,
  0.33082666666666666,
  0.45184213333333334,
  0.4619898666666667,
  0.5830112000000001],
 [[True, True, False, False],
  [True, True, False, True],
  [True, True, True, True],
  [True, True, True, False],
  [True, True, False, False],
  [True, True, False, True],
  [True, True, True, True],
  [True, True, True, False],
  [True, True, False, False],
  [True, True, False,

In [26]:
max_data

0.004221904091536999